## Week 3 MA544
---
**Objectives and Plan**

1. Pseudo-inverse (Perron-Frobenius inverse) Properties
1. Linear Systems of Equations and Gaussian Elimination with pivoting
1. LU Decomposition of A
1. QR Decomposition of a matrix
1. Iterative solution of Linear Systems

In [4]:

#IMPORT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Set a seed for the random number generator
np.random.seed(100)

## Linear System of Equations
---
Consider the following system of $m$ linear equations in $n$ variables.
\begin{align}
a_{11} x_1 + a_{12} x_2  + \cdots + a_{1n} x_n  &= b_1 \\
a_{21} x_1 + a_{22} x_2  + \cdots + a_{2n} x_n  &= b_2 \\
 \vdots \qquad \qquad   & \ \\
a_{m1} x_1 + a_{m2} x_2  + \cdots + a_{mn} x_n  &= b_ m,
\end{align}

-  The solution of a linear system represents the **point of intersection of hyperplanes**

$$
 x_1 \begin{pmatrix}a_{11}\\a_{21}\\ \vdots \\a_{m1}\end{pmatrix} +
 x_2 \begin{pmatrix}a_{12}\\a_{22}\\ \vdots \\a_{m2}\end{pmatrix} +
 \cdots +
 x_n \begin{pmatrix}a_{1n}\\a_{2n}\\ \vdots \\a_{mn}\end{pmatrix}
 =
 \begin{pmatrix}b_1\\b_2\\ \vdots \\b_m\end{pmatrix}
 $$
 
 
-  The solution also represents the **linear coding of the columns** of a matrix $A$ to get a vector $b$ in the column space ($\mathcal{R}(A)$).
 
- The system could be represented in a compact form as $Ax = b$, where 
 $$
 A=
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{pmatrix},\quad
\mathbf{x}=
\begin{pmatrix}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{pmatrix},\quad
\mathbf{b}=
\begin{pmatrix}
b_1 \\
b_2 \\
\vdots \\
b_m
\end{pmatrix}
$$

>- The system is called...
>>- overdetermined when $m > n$.
>>- underdetermined when $ m < n$.

## Gaussian elimination with scaled-row partial pivoting
---
>- What is the need for pivoting?
>- How can you find a factorization of the matrix by using the followign code.
>- How can you modify the code to find the determinant of a matrix?
>- How can you modify the following code to solve a linear system?
>- How can you modify the code to find the inverse of a matrix?


In [5]:
## Gaussian Elimination: Scaled Row Pivoting
## This function is based on the pseudo-code on page-148 in the Text by Kincaid and Cheney
def GE_rsp(A):
    '''
    This function returns the P'LU factorization of a square matrix A
    by scaled row partial pivoting. 
    In place of returning L and U, elements of modified A are used to hold values of L and U.
    '''
    m,n = A.shape
    
    L = np.eye(n) # Not being used
    U = np.zeros_like(A) # Not being used
    if m !=n:
        sys.exit("This function needs a square matrix as an input.")
        
    # The initial ordering of rows
    p = list(range(n))
    
    # Scaling vector: absolute maximum elements of each row
    s = np.max(np.abs(A), axis=1)
    
    print("Scaling Vector: ",s)
    
    # Start the k-1 passes of Guassian Elimination on A
    for k in range(n-1):
        
        print("\n PASS {}: \n".format(k+1), A)
        # Find the pivot element and interchange the rows
        pivot_index = k + np.argmax(np.abs(A[p[k:], k])/s[p[k:]])        
        
        # Interchange element in the permutation vector
        if pivot_index !=k:
            temp = p[k]
            p[k]=p[pivot_index]
            p[pivot_index] = temp
            print("permutation vector: ",p)
            
        print("\n Pivot Element: {0:.2f} \n".format(A[p[k],k]))
        if np.abs(A[p[k],k]) < 10**(-20):
             sys.exit("ERROR!! Provided matrix is singular.")
        
        # For the k-th pivot row Perform the Gaussian elimination on the following rows
        for i in range(k+1, n):
            # Find the multiplier
            z = A[p[i],k]/A[p[k],k]
            
            #Save z in A itself. You can save this in L also
            A[p[i],k] = z
            
            #Elimination operation: Changes all elements in a row simultaneously
            ##
            A[p[i],k+1:] -= z*A[p[k],k+1:]
    return A, p

In [6]:
## Example on page number 146 (Kincaid Cheney).
## Example solved manually in class
# A = np.array([[2, 3, -6], [1,-6,8], [3, -2, 1]], dtype=float)
A = np.array([[1, 6, 0], [2,1,0], [0, 2, 1]], dtype=float)
print("\n Given A: \n ",A)
A,p =GE_rsp(A)
print("\n After Gaussian Elimination with RSPP: \n", A)
print("\n The permutation Vector is: \n", p)



 Given A: 
  [[1. 6. 0.]
 [2. 1. 0.]
 [0. 2. 1.]]
Scaling Vector:  [6. 2. 2.]

 PASS 1: 
 [[1. 6. 0.]
 [2. 1. 0.]
 [0. 2. 1.]]
permutation vector:  [1, 0, 2]

 Pivot Element: 2.00 


 PASS 2: 
 [[0.5 5.5 0. ]
 [2.  1.  0. ]
 [0.  2.  1. ]]
permutation vector:  [1, 2, 0]

 Pivot Element: 2.00 


 After Gaussian Elimination with RSPP: 
 [[ 0.5   2.75 -2.75]
 [ 2.    1.    0.  ]
 [ 0.    2.    1.  ]]

 The permutation Vector is: 
 [1, 2, 0]


### LU Factorization 
---
One can use the Gaussian elimination to decompose the matrix A as follows
$$
P A = L U,\text{ or }\ A = P^T L U;
$$
where $P$ is a permutation matrix, $L$ is a unit lower-triangular matrix and $U$ is an upper triangular matrix.

In [7]:
print("\n Upper triangular, U:\n ", np.triu(A[p,:]))
print("\n Lower triangular, L:\n", np.tril(A[p,:], -1)+np.eye(3))
print("The Permutation Matrix, P:\n",np.eye(3)[p,:])


 Upper triangular, U:
  [[ 2.    1.    0.  ]
 [ 0.    2.    1.  ]
 [ 0.    0.   -2.75]]

 Lower triangular, L:
 [[1.   0.   0.  ]
 [0.   1.   0.  ]
 [0.5  2.75 1.  ]]
The Permutation Matrix, P:
 [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


## Iterative Methods for Linear Systems
---


### Jacobi Method

In [ ]:
# You can modify this code to answer the following
'''
Jacobi's iteration method for solving the system of equations Ax=b.
p0 is the initialization for the iteration.
'''
def jacobi(A, b, p0, tol, maxIter=100):
    n=len(A)
    p = p0

    for k in range(maxIter):
        p_old = p.copy() # In python assignment is not the same as copy
        
        # Update every component of iterant p
        for i in range(n):
            sumi = b[i];
            for j in range(n):
                if i==j: # Diagonal elements are not included in Jacobi
                    continue;
                sumi = sumi - A[i,j] * p_old[j]
            p[i] = sumi/A[i,i]
                
        rel_error = np.linalg.norm(p-p_old)/n
        # print("Relative error in iteration", k+1,":",rel_error)
        if rel_error<tol:
            print("TOLERANCE MET BEFORE MAX-ITERATION")
            break
    return p;

In [ ]:
# Example System
A = np.array([[10, -1, 2, 0],
              [-1, 11, -1, 3],
              [2, -1, 10, -1],
              [0, 3, -1, 8]],dtype=float)
b = np.array([6, 25, -11, 15],dtype=float)